In [ ]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt 
import numpy as np
import optuna
import numpy as np
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
import xgboost as xgb
from xgboost import XGBRegressor
from ydata_profiling import ProfileReport

import catboost as cb
from scalecast.Forecaster import Forecaster

import lightgbm as lgb


alt.data_transformers.disable_max_rows()
sklearn.set_config(transform_output="pandas")

## Import Data

In [101]:
train = pd.read_csv('train.csv')
blind = pd.read_csv('test.csv')

In [6]:
data_profile = ProfileReport(train)

In [47]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


In [142]:
def feature_eng(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])

    # Extract date features
    
    df['day_of_year'] = df['date'].dt.dayofyear
    df['day_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year

    # Encode categorical features


    # Drop missing values and irrelevant columns
    imputer = SimpleImputer(strategy="median")  # Change to "mean", "most_frequent", or "constant"
    scaler = StandardScaler()

    
    df = df.set_index('date')
    df = df.drop(columns=['id'])  # Dropping columns that are not useful for modeling
    # Separate features and target
    try: 
        df["num_sold"] = imputer.fit_transform(df[["num_sold"]])
        df['num_sold'] = scaler.fit_transform(df[['num_sold']])
    except: pass
    try: 
        X = df.drop(columns=['num_sold'])
        X = pd.get_dummies(X)

    except:
        X=pd.get_dummies(df)
    try:
        y = df['num_sold']
    except:
        y = None
    # Scale features
    #X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

    return df, X,  y

# Feature engineering
df, X, y = feature_eng(train)

# Splitting the data
SPLIT = 0.85
split_index = int(SPLIT * len(X))

X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

# Verify shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (195610, 18)
y_train shape: (195610,)
X_test shape: (34520, 18)
y_test shape: (34520,)


In [108]:
X_test.head(1)

,day_of_year,day_week,month,year,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
date,,,,,,,,,,,,,,,,,,
2015-12-14,348,0,12,2015,False,False,True,False,False,False,False,True,False,True,False,False,False,False


In [111]:
X_train

,day_of_year,day_week,month,year,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
date,,,,,,,,,,,,,,,,,,
2010-01-01,1,4,1,2010,True,False,False,False,False,False,True,False,False,True,False,False,False,False
2010-01-01,1,4,1,2010,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2010-01-01,1,4,1,2010,True,False,False,False,False,False,True,False,False,False,False,True,False,False
2010-01-01,1,4,1,2010,True,False,False,False,False,False,True,False,False,False,False,False,True,False
2010-01-01,1,4,1,2010,True,False,False,False,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-14,348,0,12,2015,False,False,True,False,False,False,False,False,True,True,False,False,False,False
2015-12-14,348,0,12,2015,False,False,True,False,False,False,False,False,True,False,True,False,False,False
2015-12-14,348,0,12,2015,False,False,True,False,False,False,False,False,True,False,False,True,False,False


In [37]:
blind, X_blind, y_blind = feature_eng(blind)

In [113]:
model = LinearRegression()

# Train the model using training data
model.fit(X_train.values, y_train.values)

# Make predictions on the test data
y_pred = model.predict(X_test.values)

# Evaluate the model using common metrics
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print(f"Mean Absolute Error: {mae:.4f}")
print(f"MAPE: {mape:.4f}")

print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")

# Optionally: You can get the coefficients and intercept of the linear regression model
print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

Mean Absolute Error: 0.3537
MAPE: 2.0032
Mean Squared Error: 0.2389
Root Mean Squared Error: 0.4888
R-squared: 0.6740
Coefficients: [ 3.20350518e-05  3.51845565e-02 -2.34001964e-03 -9.43767662e-03
  1.78030297e+10  1.78030297e+10  1.78030297e+10  1.78030297e+10
  1.78030297e+10  1.78030297e+10 -1.05459943e+07 -1.05459935e+07
 -1.05459937e+07 -1.16599403e+08 -1.16599401e+08 -1.16599402e+08
 -1.16599402e+08 -1.16599402e+08]
Intercept: -17675884326.186516


c:\Users\Leo\.pyenv\pyenv-win-venv\envs\venv_kaggle\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [127]:
def objective(trial):
    print(f"🔄 Starting Trial {trial.number + 1} out of {20}")  # Replace 20 with your actual n_trials

    # Define hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0),
        "enable_categorical": True,  # Enable categorical features
        "device": "gpu",  # Ensure GPU usage
        "tree_method": "hist"  # Recommended for GPU acceleration in XGBoost 2.0+
    }

    # Train the model
    model = xgb.XGBRegressor(**param)
    model.fit(X_train.values, y_train.values)

    # Predict & compute MAPE
    preds = model.predict(X_test.values)
    mape = mean_absolute_percentage_error(y_test.values, preds)
    #blind_preds = model.predict(blind)
    #blind_mape = mean_absolute_percentage_error(y_train, blind_preds)
    print(f"✅ Completed Trial {trial.number + 1} - MAPE: {mape:.4f}")

    return mape

# Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)  # ✅ Use explicit n_trials here


[I 2025-01-30 10:35:53,114] A new study created in memory with name: no-name-198e2c31-ba27-48f0-9a1c-297b3c1e1593


🔄 Starting Trial 1 out of 20


[I 2025-01-30 10:35:54,883] Trial 0 finished with value: 0.5758351736151526 and parameters: {'n_estimators': 130, 'learning_rate': 0.10943253983197812, 'max_depth': 11, 'subsample': 0.6013504973775846, 'colsample_bytree': 0.5906589526889034, 'reg_alpha': 3.2710342017639675, 'reg_lambda': 7.7169692595115675}. Best is trial 0 with value: 0.5758351736151526.


✅ Completed Trial 1 - MAPE: 0.5758
🔄 Starting Trial 2 out of 20


[I 2025-01-30 10:35:57,111] Trial 1 finished with value: 0.6069061455511644 and parameters: {'n_estimators': 187, 'learning_rate': 0.018836644317455076, 'max_depth': 17, 'subsample': 0.9946018258495533, 'colsample_bytree': 0.6300119199867369, 'reg_alpha': 8.857130257295983, 'reg_lambda': 9.904518856531032}. Best is trial 0 with value: 0.5758351736151526.


...

✅ Completed Trial 3 - MAPE: 0.5710
🔄 Starting Trial 4 out of 20


[I 2025-01-30 10:36:00,884] Trial 3 finished with value: 0.5382838473338825 and parameters: {'n_estimators': 61, 'learning_rate': 0.0687564554580413, 'max_depth': 17, 'subsample': 0.7829001072816584, 'colsample_bytree': 0.916508052610321, 'reg_alpha': 8.052439257010953, 'reg_lambda': 0.8988330904106796}. Best is trial 3 with value: 0.5382838473338825.


✅ Completed Trial 4 - MAPE: 0.5383
🔄 Starting Trial 5 out of 20


[I 2025-01-30 10:36:02,136] Trial 4 finished with value: 0.7596135994572786 and parameters: {'n_estimators': 72, 'learning_rate': 0.011081967714121278, 'max_depth': 18, 'subsample': 0.9118912293778543, 'colsample_bytree': 0.7068779377782985, 'reg_alpha': 6.777399471532983, 'reg_lambda': 8.345889783452982}. Best is trial 3 with value: 0.5382838473338825.


✅ Completed Trial 5 - MAPE: 0.7596


In [118]:
y_test

date
2015-12-14   -0.851643
2015-12-14    0.443153
2015-12-14    0.345711
2015-12-14   -0.345240
2015-12-14   -0.376245
                ...   
2016-12-31   -0.414631
2016-12-31    3.189243
2016-12-31    2.291596
2016-12-31    0.731050
2016-12-31    1.292079
Name: num_sold, Length: 34520, dtype: float64

In [119]:
#Catboost
def objective_cb(trial):
    print(f"🔄 Starting CatBoost Trial {trial.number + 1} out of {20}") 

    param = {
        "iterations": trial.suggest_int("iterations", 50, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "depth": trial.suggest_int("depth", 3, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-5, 10.0, log=True),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "Poisson"]),
        "task_type": "GPU",  # Enable GPU usage
        "devices": "0"
    }

    # Ensure subsample is only used when bootstrap_type supports it
    if param["bootstrap_type"] in ["Bernoulli", "Poisson"]:
        param["subsample"] = trial.suggest_float("subsample", 0.5, 1.0)

    model = cb.CatBoostRegressor(**param, verbose=0)
    model.fit(X_train.values, y_train.values)

    preds = model.predict(X_test.values)
    mape = mean_absolute_percentage_error(y_test.values, preds)

    print(f"✅ Completed CatBoost Trial {trial.number + 1} - MAPE: {mape:.4f}")
    return mape

# Run Optuna study for CatBoost
study_cb = optuna.create_study(direction="minimize")
study_cb.optimize(objective_cb, n_trials=5)


[I 2025-01-30 10:32:17,429] A new study created in memory with name: no-name-ca9d989e-e7d7-4a3b-ba50-cfc0531a8b02


🔄 Starting CatBoost Trial 1 out of 20


[I 2025-01-30 10:32:19,864] Trial 0 finished with value: 0.5642048639613763 and parameters: {'iterations': 110, 'learning_rate': 0.07761315610356374, 'depth': 10, 'l2_leaf_reg': 1.287032171603146e-05, 'bootstrap_type': 'Bernoulli', 'subsample': 0.993958007682485}. Best is trial 0 with value: 0.5642048639613763.


✅ Completed CatBoost Trial 1 - MAPE: 0.5642
🔄 Starting CatBoost Trial 2 out of 20


[I 2025-01-30 10:32:21,459] Trial 1 finished with value: 1.2054574011116603 and parameters: {'iterations': 183, 'learning_rate': 0.045263313626500004, 'depth': 3, 'l2_leaf_reg': 0.0005504960257199449, 'bootstrap_type': 'Bayesian'}. Best is trial 0 with value: 0.5642048639613763.


...

✅ Completed CatBoost Trial 3 - MAPE: 0.6217
🔄 Starting CatBoost Trial 4 out of 20


[I 2025-01-30 10:32:24,749] Trial 3 finished with value: 1.3690629686356102 and parameters: {'iterations': 135, 'learning_rate': 0.013700592645527324, 'depth': 5, 'l2_leaf_reg': 7.120787883299445e-05, 'bootstrap_type': 'Poisson', 'subsample': 0.7280860114280876}. Best is trial 0 with value: 0.5642048639613763.


✅ Completed CatBoost Trial 4 - MAPE: 1.3691
🔄 Starting CatBoost Trial 5 out of 20


[I 2025-01-30 10:32:26,275] Trial 4 finished with value: 0.6745534215792119 and parameters: {'iterations': 63, 'learning_rate': 0.0703410258316165, 'depth': 8, 'l2_leaf_reg': 0.03814485996533083, 'bootstrap_type': 'Bernoulli', 'subsample': 0.6321310790982253}. Best is trial 0 with value: 0.5642048639613763.


✅ Completed CatBoost Trial 5 - MAPE: 0.6746


In [120]:
#lightGBM
import lightgbm as lgb

def objective_lgb(trial):
    print(f"🔄 Starting Trial {trial.number + 1} out of {20}")

    param = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 18),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10.0),
        "device": "gpu",  # Enable GPU acceleration
        "boosting_type": "gbdt"
    }

    # Train the model
    model = lgb.LGBMRegressor(**param)
    model.fit(X_train.values, y_train.values)

    # Predict & compute MAPE
    preds = model.predict(X_test.values)
    mape = mean_absolute_percentage_error(y_test.values, preds)
    
    print(f"✅ Completed Trial {trial.number + 1} - MAPE: {mape:.4f}")

    return mape

# Run Optuna study for LightGBM
study_lgb = optuna.create_study(direction="minimize")
study_lgb.optimize(objective_lgb, n_trials=5)


[I 2025-01-30 10:32:53,263] A new study created in memory with name: no-name-59249260-7cad-411a-87c9-b392b930b2cf


🔄 Starting Trial 1 out of 20
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 195610, number of used features: 18
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1660 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (1.49 MB) transferred to GPU in 0.004394 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.018130
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

[I 2025-01-30 10:32:55,016] Trial 0 finished with value: 0.6661336500835574 and parameters: {'n_estimators': 143, 'learning_rate': 0.05421579704191731, 'max_depth': 5, 'subsample': 0.7714021918077434, 'colsample_bytree': 0.7757299648452833, 'reg_alpha': 6.666265064362723, 'reg_lambda': 7.404644607747894}. Best is trial 0 with value: 0.6661336500835574.


✅ Completed Trial 1 - MAPE: 0.6661
🔄 Starting Trial 2 out of 20
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 195610, number of used features: 18
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1660 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (1.49 MB) transferred to GPU in 0.003377 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.018130
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2025-01-30 10:32:56,419] Trial 1 finished with value: 0.5890301337015383 and parameters: {'n_estimators': 120, 'learning_rate': 0.23917404264091283, 'max_depth': 6, 'subsample': 0.5702624363564109, 'colsample_bytree': 0.5302540189053162, 'reg_alpha': 9.181284904224954, 'reg_lambda': 5.777833963385993}. Best is trial 1 with value: 0.5890301337015383.


...

✅ Completed Trial 3 - MAPE: 0.6306
🔄 Starting Trial 4 out of 20
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 195610, number of used features: 18
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1660 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (1.49 MB) transferred to GPU in 0.003020 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.018130
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2025-01-30 10:32:59,950] Trial 3 finished with value: 0.6655998943569493 and parameters: {'n_estimators': 100, 'learning_rate': 0.20510095694932834, 'max_depth': 15, 'subsample': 0.6958803783553931, 'colsample_bytree': 0.7942416619544808, 'reg_alpha': 8.023697065904198, 'reg_lambda': 1.432326646475349}. Best is trial 1 with value: 0.5890301337015383.


✅ Completed Trial 4 - MAPE: 0.6656
🔄 Starting Trial 5 out of 20
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 195610, number of used features: 18
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1660 Ti, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 7 dense feature groups (1.49 MB) transferred to GPU in 0.004067 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 0.018130
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

[I 2025-01-30 10:33:01,786] Trial 4 finished with value: 0.6758076443891302 and parameters: {'n_estimators': 135, 'learning_rate': 0.06539363345497198, 'max_depth': 5, 'subsample': 0.7668706791010944, 'colsample_bytree': 0.7899188132034961, 'reg_alpha': 9.583677812457712, 'reg_lambda': 0.7563834387705128}. Best is trial 1 with value: 0.5890301337015383.


✅ Completed Trial 5 - MAPE: 0.6758


In [121]:
blind

,id,date,country,store,product
0,230130,2017-01-01,Canada,Discount Stickers,Holographic Goose
1,230131,2017-01-01,Canada,Discount Stickers,Kaggle
2,230132,2017-01-01,Canada,Discount Stickers,Kaggle Tiers
3,230133,2017-01-01,Canada,Discount Stickers,Kerneler
4,230134,2017-01-01,Canada,Discount Stickers,Kerneler Dark Mode
...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler


In [143]:
X_trial = feature_eng(blind)[1]


In [144]:
X_trial

,day_of_year,day_week,month,year,country_Canada,country_Finland,country_Italy,country_Kenya,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
date,,,,,,,,,,,,,,,,,,
2017-01-01,1,6,1,2017,True,False,False,False,False,False,True,False,False,True,False,False,False,False
2017-01-01,1,6,1,2017,True,False,False,False,False,False,True,False,False,False,True,False,False,False
2017-01-01,1,6,1,2017,True,False,False,False,False,False,True,False,False,False,False,True,False,False
2017-01-01,1,6,1,2017,True,False,False,False,False,False,True,False,False,False,False,False,True,False
2017-01-01,1,6,1,2017,True,False,False,False,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31,365,1,12,2019,False,False,False,False,False,True,False,True,False,True,False,False,False,False
2019-12-31,365,1,12,2019,False,False,False,False,False,True,False,True,False,False,True,False,False,False
2019-12-31,365,1,12,2019,False,False,False,False,False,True,False,True,False,False,False,True,False,False


In [129]:
study.best_params

{'n_estimators': 61,
 'learning_rate': 0.0687564554580413,
 'max_depth': 17,
 'subsample': 0.7829001072816584,
 'colsample_bytree': 0.916508052610321,
 'reg_alpha': 8.052439257010953,
 'reg_lambda': 0.8988330904106796}

In [137]:
best_params = study.best_params
final_model = xgb.XGBRegressor(enable_categorical=True, 
                               device="gpu",
                               **best_params)
final_model.fit(X_train.values, y_train.values)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.916508052610321, device='gpu',
             early_stopping_rounds=None, enable_categorical=True,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.0687564554580413, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=17, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=61, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

array([-0.20920183,  0.10042469, -0.09251786, ...,  1.5534148 ,
        0.32471746,  0.5156916 ], dtype=float32)

In [152]:
predictions = final_model.predict(X_trial.values)
submission_df = pd.DataFrame({"predictions": predictions})
submission_df = pd.merge(blind['id'], 
         submission_df,
         left_index=True,
         right_index=True)
submission_file = submission_df.to_csv("submission.csv", index=False)


In [151]:
#.to_csv('submission.csv', index=False)

,id,predictions
0,230130,-0.209202
1,230131,0.100425
2,230132,-0.092518
3,230133,-0.471277
4,230134,-0.370374
...,...,...
98545,328675,-0.528569
98546,328676,2.157916
98547,328677,1.553415
98548,328678,0.324717


In [149]:
submission_df

,predictions
0,-0.209202
1,0.100425
2,-0.092518
3,-0.471277
4,-0.370374
...,...
98545,-0.528569
98546,2.157916
98547,1.553415
98548,0.324717


In [ ]:
blind_predition_model = model.fit(test)

NameError: name 'model' is not defined